# SSMR: generate SSM model from trajectory data

In [167]:
import numpy as np
from scipy.io import loadmat
from copy import deepcopy
import os
from os.path import join, exists, isdir
import pickle

In [168]:
%matplotlib qt
import matplotlib.pyplot as plt

In [169]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
np.set_printoptions(linewidth=300)

In [171]:
use_data = "john" # "mine", "john"
observables = "delay embedding" #"pos-vel" # "delay embedding"
TIP_NODE = 1354

## Import and inspect data
See mat file in URL

In [184]:
Data = {}
if use_data == "john":
    dataDecayObs = loadmat("../dataDecayObs.mat") # , simplify_cells=True)
    Data['oData'] = dataDecayObs['oData'].tolist()
    Data['etaData'] = dataDecayObs['yData'].tolist()
    # Xsnapshots = dataDecayObs['Xsnapshots']
    for i in range(len(Data['oData'])):
        Data['oData'][i][0] = Data['oData'][i][0].squeeze()
        # Data['yData'][i][0] = Data['yData'][i][0].squeeze()
    data_dir = "/home/jalora/Desktop/diamond"
    with open(join(data_dir, "rest_q.pkl"), "rb") as f:
        q_eq = np.array(pickle.load(f))
    
    # diamond_dir = "/home/jalora/Desktop/diamond"
    # with open(join(diamond_dir, "rest_q.pkl"), "rb") as f:
    #     q_eq = np.array(pickle.load(f))

    # decayData_dir = os.path.join(diamond_dir, "decay")
    # Data['oData'] = utils.import_pos_data(decayData_dir, q_rest=q_eq, output_node=TIP_NODE, 
    #                                       file_type='mat', subsample=1, return_velocity=True)
elif use_data == "john_full":
    pass
elif use_data == "mine":
    diamond_dir = "../../../soft-robot-control/examples/diamond"
    decayData_dir = os.path.join(diamond_dir, "dataCollection/origin")
    rest_file = os.path.join(diamond_dir, 'rest.pkl')
    Data['oData'] = utils.import_pos_data(decayData_dir, rest_file, output_node='all', t_in=2, t_out=5)

rDOF = 3
oDOF = 3
nTRAJ = len(Data['oData'])
SSMDim = 6

print("nTRAJ:", nTRAJ)

nTRAJ: 52


In [185]:
Data['oData'][0][1]
# q_eq

array([[-5.45392047e-01, -5.41038065e-01, -5.36691867e-01, ..., -2.79422117e-05, -1.94690695e-05, -1.10524528e-05],
       [ 6.50401955e+01,  6.47968882e+01,  6.44540634e+01, ...,  1.05313446e-03,  1.03745066e-03,  1.02156337e-03],
       [-2.19193508e+01, -2.21081819e+01, -2.23069720e+01, ...,  9.33464275e-05,  8.58758604e-05,  7.84392052e-05],
       [ 0.00000000e+00,  4.35398182e+00,  4.34619781e+00, ...,  8.52736312e-03,  8.47314220e-03,  8.41661673e-03],
       [ 0.00000000e+00, -2.43307287e+02, -3.42824832e+02, ..., -1.54741008e-02, -1.56838000e-02, -1.58872951e-02],
       [ 0.00000000e+00, -1.88831037e+02, -1.98790155e+02, ..., -7.50214218e-03, -7.47056704e-03, -7.43665525e-03]])

In [80]:
# if use_data == 'john':
#     for i in range(nTRAJ):
#         Data['oData'][i][0] = Data['oData'][i][0][::10]
#         Data['oData'][i][1] = Data['oData'][i][1][:, ::10]

## Observables

In [87]:
observables = "delay embedding" #pos-vel

EITHER compute reduced coordinates using delay embedding on the available oData

In [88]:
if observables == "delay embedding":
    N_DELAY = 4
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        delay_embedded_traj = utils.delayEmbedding(Data['oData'][i][1], up_to_delay=N_DELAY)
        Data['yData'][i][1] = delay_embedded_traj[:, :-4]
        Data['yData'][i][0] = Data['yData'][i][0][4:]

OR use position and velocity of all nodes as observables

In [89]:
if observables == "pos-vel":
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        # observables y are pos and vel of tip
        Data['yData'][i][1] = np.vstack((Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], np.gradient(Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], 0.01, axis=1)))
        # do SVD on full state + full velocity
        # Data['oData'][i][1] = np.vstack((Data['oData'][i][1], np.gradient(Data['oData'][i][1], axis=1)))
    print(Data['yData'][0][1].shape)

### Plot observables of interest

In [90]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [14, 16, 18, 31, 35, 44] # []
plot.traj_xyz(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[0]+oDOF), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

In [91]:
t_interval = [0.08, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [92]:
# print(Data['yDataTrunc'][0][1].shape)
# Data['yDataTrunc'][0][1][:, :5]

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [93]:
# Perform SVD on displacement field
from scipy.sparse.linalg import svds

if observables == "pos-vel":
    svd_data = 'oDataTrunc'
elif observables == "delay embedding":
    svd_data = 'yDataTrunc'

Xsnapshots = np.hstack([DataTrunc[1] for DataTrunc in Data[svd_data]])
Xsnapshots = Xsnapshots[:, :]

# print(Xsnapshots.shape)
# print(Xsnapshots[:, 0:10])

show_modes = 9
v, s, u = svds(Xsnapshots, k=max(SSMDim, show_modes), which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
# print(s)
# print(v)
l2vals = s**2

In [95]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

### Project delay-embedded data onto dominant modes

In [96]:
from copy import deepcopy
Data['etaDataTrunc'] = deepcopy(Data[svd_data])
if observables == "delay embedding":
    Vde = v[:, :SSMDim]
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ Data[svd_data][i][1]
elif observables == "pos-vel":
    assert SSMDim % 2 == 0
    Vde = np.kron(np.eye(2), v[:, :SSMDim//2])
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ np.vstack((Data['oDataTrunc'][i][1], np.gradient(Data['oDataTrunc'][i][1], 0.01, axis=1))) # np.vstack((np.gradient(Data['oData'][i][1], axis=1), Data['oData'][i][1])) # Vde.T @ np.vstack((Data[svd_data][i][1], np.gradient(Data[svd_data][i][1], axis=1))) # 

In [162]:
Vde

array([[-9.65209965e-02,  1.09133132e-01,  4.59807591e-01, -2.07464296e-01,  2.93985374e-01, -5.07962169e-01],
       [ 5.68103971e-02,  4.66852590e-01, -9.53449056e-02,  1.65004583e-01,  5.46752295e-01,  2.51142247e-01],
       [-4.43660541e-01,  4.12032627e-02, -1.02830797e-01, -5.79749534e-01,  4.50031030e-02,  2.61962805e-01],
       [-9.62131090e-02,  1.03317110e-01,  4.55153106e-01, -8.77901721e-02,  1.28612704e-01, -2.20412955e-01],
       [ 5.52363146e-02,  4.63353640e-01, -9.13637058e-02,  7.47810792e-02,  2.36033032e-01,  1.12463235e-01],
       [-4.49523871e-01,  3.57231396e-02, -1.01683373e-01, -2.67214834e-01,  2.25213227e-02,  1.22726685e-01],
       [-9.30669650e-02,  9.47471656e-02,  4.37932411e-01,  2.32546369e-02, -2.62890752e-02,  5.10356051e-02],
       [ 5.17852427e-02,  4.46813702e-01, -8.55222861e-02, -8.39271532e-03, -5.63390377e-02, -1.96420784e-02],
       [-4.44514824e-01,  3.06246478e-02, -9.97596122e-02,  3.38458608e-02,  8.02796062e-05, -1.33452961e-02],
 

In [97]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_3D(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 3), ('etaDataTrunc', 1)],
              xyz_names=[r'$x_1$', r'$\dot{x_1}$', r'$x_2$'])

### Train and test data (train/test split)

In [98]:
indTest = [0, 4] #[0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearn

Start Matlab engine and install/run SSMLearn

In [99]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jalora/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [101]:
# make data ready for Matlab
yDataTruncTrain_matlab = [[], []]
etaDataTruncTrain_matlab = [[], []]
for i in indTrain:
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))
    etaDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['etaDataTrunc'][i][0].tolist()))
    etaDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['etaDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [102]:
SSMOrder = 3
obsDim = 2*oDOF

IMInfo = eng.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', etaDataTruncTrain_matlab) # , 'l_vals', 1.)
if observables == "pos-vel":
    IMInfoInv = eng.IMGeometry(etaDataTruncTrain_matlab, obsDim, SSMOrder,
                            'reducedCoordinates', yDataTruncTrain_matlab)
    for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
        IMInfo['chart'][key] = IMInfoInv['parametrization'][key]


Data seems to have been fed from Python!


### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [103]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')

Data seems to have been fed from Python!
Estimation of the reduced dynamics... 
 Done. 


Stop Matlab engine -- not needed anymore

In [104]:
eng.quit()

Convert all matlab double arrays to numpy arrays

In [105]:
def convert_matlab_arrays_to_numpy_arrays(info_dict):
  for k,v in info_dict.items():        
     if isinstance(v, dict):
         convert_matlab_arrays_to_numpy_arrays(v)
     else:
        if isinstance(v, matlab.double):
           info_dict[k] = np.array(v)

convert_matlab_arrays_to_numpy_arrays(IMInfo)
convert_matlab_arrays_to_numpy_arrays(RDInfo)

Stability analysis of reduced dynamics

In [106]:
assert np.all(np.real(RDInfo['eigenvaluesLinPartFlow']) < 0)
RDInfo['eigenvaluesLinPartFlow']

array([[-3.63442053+16.74040193j],
       [-3.75848629+17.07569491j],
       [-3.6287267 +18.15614517j],
       [-3.63442053-16.74040193j],
       [-3.75848629-17.07569491j],
       [-3.6287267 -18.15614517j]])

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [107]:
yRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [108]:
yRec['Train'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(yRec['Train'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")

Average parametrization train error: 2.2757e-02


#### Test error

In [109]:
yRec['Test'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(yRec['Test'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Average parametrization test error: 2.5965e-02


Plot comparison of SSM-predicted vs. actual test trajectories

In [110]:
print(Data['yData'][0][1].shape)
print(yRec['Test'][0][1].shape)

(15, 296)
(15, 252)


In [111]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [112]:
etaRec = {}
etaRec['RD'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
meanErrorDyn = {}

for i in range(nTRAJ):
    if etaRec['RD'][i][1].shape[1] != Data['etaDataTrunc'][i][1].shape[1]:
        print(i)

normedTrajDist = utils.compute_trajectory_errors(etaRec['RD'], Data['etaDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

5
Average dynamics train error: inf
Average dynamics test error: 2.3174


In [113]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(etaRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [114]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [115]:
yRec['RDTrain'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(yRec['RDTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: inf


#### Test error

In [116]:
yRec['RDTest'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(yRec['RDTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 2.3198


In [117]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

## Control

### Setup model for control

In [118]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [119]:
Rauton = utils.Rauton(RDInfo)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
if observables == "pos-vel":
    Wauton = lambda y: IMInfo['chart']['H'] @ utils.multivariate_polynomial(y, IMInfo['chart']['polynomialOrder'])
elif observables == "delay embedding":
    Wauton = lambda y: Vde.T @ y

In [160]:
IMInfo['parametrization']['H']

array([[-9.74058382e-02,  1.09312209e-01,  4.58848543e-01, ...,  6.84570219e-07, -7.43715667e-06,  1.82850325e-06],
       [ 5.68678586e-02,  4.66453419e-01, -9.47683674e-02, ..., -6.80896529e-06, -8.43778278e-08, -1.35244143e-06],
       [-4.39824349e-01,  4.09380936e-02, -9.98648085e-02, ..., -1.77309586e-06,  5.03693537e-06,  9.42426708e-06],
       ...,
       [-8.01424658e-02,  7.13231426e-02,  3.69881843e-01, ...,  7.07813562e-07, -5.08978366e-06,  2.33402784e-06],
       [ 4.00373585e-02,  3.79199628e-01, -6.85219650e-02, ..., -4.55078952e-06, -2.98613295e-06, -2.08196102e-06],
       [-4.02193690e-01,  2.10319669e-02, -9.03021420e-02, ..., -2.51077560e-06,  6.14316927e-06,  7.06563535e-06]])

### Learn control matrix $B$

Use figure8 trajectory as the training data

In [120]:
# # Load the controls and actual trajectories for figure8
# u_train = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
# z_train = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
# z_eq_train = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
# print(z_eq_train)

# # tip position, normalized using resting position
# z_shift_train = (z_train[-3:, :].T - z_eq_train).T

# # original spacing of inputs
# T = 10.01
# dt = 0.01
# N = int(T / dt)
# t_train = np.linspace(0, T, N+1)

Use randomly sampled (step) inputs as the training data

In [121]:
# diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
# z_train, u_train, _ = diamond_train_full['y'].T, diamond_train_full['u'].T, diamond_train_full['t']
# print(z_train.shape)

# # tip position, normalized using resting position
# z_shift_train = (z_train[-3:, :].T - z_train[-3:, 0]).T
# dt = 0.01
# t_train = np.arange(z_train.shape[1]) * dt

Use randomly sampled inputs as the training and test data

In [139]:
testInputDir = "/home/jalora/Desktop/diamond/open-loop/"
(t, z_shift), u = utils.import_pos_data(data_dir=testInputDir,
                                    q_rest=q_eq, # join(SETTINGS['robot_dir'], SETTINGS['rest_file']),
                                    output_node=1354, return_inputs=True, file_type='mat')
# print(u_train)
dt = 0.01

In [161]:
z_shift

array([[-9.65209965e-02,  1.09133132e-01,  4.59807591e-01, -2.07464296e-01,  2.93985374e-01, -5.07962169e-01],
       [ 5.68103971e-02,  4.66852590e-01, -9.53449056e-02,  1.65004583e-01,  5.46752295e-01,  2.51142247e-01],
       [-4.43660541e-01,  4.12032627e-02, -1.02830797e-01, -5.79749534e-01,  4.50031030e-02,  2.61962805e-01],
       [-9.62131090e-02,  1.03317110e-01,  4.55153106e-01, -8.77901721e-02,  1.28612704e-01, -2.20412955e-01],
       [ 5.52363146e-02,  4.63353640e-01, -9.13637058e-02,  7.47810792e-02,  2.36033032e-01,  1.12463235e-01],
       [-4.49523871e-01,  3.57231396e-02, -1.01683373e-01, -2.67214834e-01,  2.25213227e-02,  1.22726685e-01],
       [-9.30669650e-02,  9.47471656e-02,  4.37932411e-01,  2.32546369e-02, -2.62890752e-02,  5.10356051e-02],
       [ 5.17852427e-02,  4.46813702e-01, -8.55222861e-02, -8.39271532e-03, -5.63390377e-02, -1.96420784e-02],
       [-4.44514824e-01,  3.06246478e-02, -9.97596122e-02,  3.38458608e-02,  8.02796062e-05, -1.33452961e-02],
 

Train/test split

In [141]:
train_ratio = 1.0
split_idx = int(train_ratio * len(t))
t_train, t_test = t[:split_idx], t[split_idx:]
z_shift_train, z_shift_test = z_shift[:, :split_idx], z_shift[:, split_idx:]
u_train, u_test = u[:, :split_idx], u[:, split_idx:]

Plot the training data

In [142]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Plot the test data

In [143]:
plt.close('all')
plot.traj_xyz_txyz(t=t_test,
                   x=z_shift_test[0, :], y=z_shift_test[1, :], z=z_shift_test[2, :])
plot.inputs(t_test, u_test)

In [144]:
z_shift_train.shape

(3, 8692)

Do delay embedding on the tip position. Then, compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [145]:
if observables == "delay embedding":
    # delay-embed tip position
    y_train = utils.delayEmbedding(z_shift_train, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_train = np.vstack((z_shift_train, np.gradient(z_shift_train, 0.01, axis=1)))

print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
x_train = Wauton(y_train)
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
keep_indices = np.s_[10:-10]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train_trunc.shape)

y.shape: (15, 8692), u.shape: (4, 8692)
(8672,)


In [146]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

Plot gradients (numerical differentiation and reduced dynamics)

In [147]:
print(dxDt.shape)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

(6, 8672)


In [148]:
plt.close('all')
plot.dependence_of_xdot_on_inputs(dxDt, u_train_trunc)

Learn $B$ matrix

In [149]:
from sklearn.linear_model import Ridge

polyUorder = 1
# assemble_features = lambda u, x: utils.multivariate_polynomial(np.vstack([u, x]), order=polyUorder)
assemble_features = lambda u, x: utils.multivariate_polynomial(u, order=polyUorder)

X = assemble_features(u_train_trunc, x_train_trunc)
print(X.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
ridgeModel.fit(X.T, (dxDt - dxDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

(4, 8672)
(6, 4)


Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [150]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Frobenius norm of B_learn: 0.3358


Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [151]:
plt.close('all')
plot_reduced_coords = np.s_[:] # [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train_trunc) + B_learn @ assemble_features(u_train_trunc, x_train_trunc) # utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [152]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs (big amplitude) -- and using learned influence of control

Try to predict the test trajectory using reduced dynamics

In [153]:
if observables == "delay embedding":
    # delay-embed tip position
    y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_test = np.vstack((z_shift_test, np.gradient(z_shift_test, 0.01, axis=1)))

# reduced coordinates
x_test = Wauton(y_test)

slice_traj = np.s_[:]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_shift_test = z_shift_test[:, slice_traj]

In [154]:
# Load the controls and actual trajectories for figure8
u_test = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_test = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_test = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_test.shape)
# tip position, normalized using resting position
z_shift_test = (z_test[-3:, :].T - z_eq_test).T
# delay-embed tip position (as well as inputs?)
if observables == "delay embedding":
    # delay-embed tip position
    y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_test = np.vstack((z_shift_test, np.gradient(z_shift_test, 0.01, axis=1)))# reduced coordinates
x_test = Wauton(y_test)

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_test = np.linspace(0, T, N+1)

slice_traj = np.s_[:]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_test = z_test[:, slice_traj]

(6, 1002)


In [155]:
uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ assemble_features(uFun(t), y) # utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = yTraj[-3:, :] # .T + z_eq_test).T

plt.close('all')
axs = plot.traj_xyz_txyz(t_test,
                    zTraj[outdofs[0], :],
                    zTraj[outdofs[1], :],
                    zTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_test[3, :],
                      z_test[4, :],
                      z_test[5, :], color="tab:orange", ax=ax, show=False)
axs = plot.traj_xyz_txyz(t_test,
                    z_shift_test[outdofs[0], :],
                    z_shift_test[outdofs[1], :],
                    z_shift_test[outdofs[2], :], color="tab:orange", axs=axs, show=False)
axs[2].legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [156]:
zTraj

array([[-0.20483557, -0.31059362, -0.43380844, ..., -0.47495433, -0.53280607, -0.59256425],
       [ 0.05844582,  0.08423716,  0.14164297, ..., -0.94585978, -0.72714141, -0.50472842],
       [ 1.30198535,  2.02116091,  2.83587861, ...,  6.29379323,  6.22865088,  6.16152458]])

In [157]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_shift_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 0.5460


## Save SSM model

Continuous model

In [125]:
# SSM_model = {'model': {}, 'params': {}}

# print(IMInfo['parametrization']['H'].shape)
# print(IMInfo['chart']['H'].shape)
# SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
# SSM_model['model']['v_coeff'] = IMInfo['chart']['H']
# SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients']
# SSM_model['model']['B'] = B_learn
# SSM_model['model']['V'] = None # Vde

# SSM_model['params']['SSM_order'] = SSMOrder
# SSM_model['params']['ROM_order'] = ROMOrder
# SSM_model['params']['state_dim'] = SSMDim # 2 * rDOF
# print(SSMDim)
# # print(2*rDOF)
# SSM_model['params']['input_dim'] = B_learn.shape[1]
# SSM_model['params']['output_dim'] = oDOF * 2

# if observables == "delay embeddings":
#     SSM_model['params']['delays'] = N_DELAY
#     SSM_model['params']['obs_dim'] = 3 * (1 + N_DELAY)
# elif observables == "pos-vel":
#     SSM_model['params']['delays'] = 0
#     SSM_model['params']['obs_dim'] = 6
# # print(Data['yData'][0][1].shape[0])

In [126]:
# import pickle
# # save dictionary to person_data.pkl file
# with open('../../../soft-robot-control/examples/hardware/SSMmodels/SSM_model.pkl', 'wb') as f:
#     pickle.dump(SSM_model, f)